importaation et chemin 

In [5]:
from pathlib import Path
import pandas as pd
import numpy as np

BASE_DIR = Path(r"C:\Users\Brydel\Desktop\RUL_Predictor")
RAW_DIR  = BASE_DIR / "data" / "raw"
PROC_DIR = BASE_DIR / "data" / "processed"
DOCS_DIR = BASE_DIR / "docs"

PROC_DIR.mkdir(parents=True, exist_ok=True)
DOCS_DIR.mkdir(parents=True, exist_ok=True)

train_path = RAW_DIR / "train_FD001.txt"
test_path  = RAW_DIR / "test_FD001.txt"
rul_path   = RAW_DIR / "RUL_FD001.txt"

train_path.exists(), test_path.exists(), rul_path.exists()


(True, True, True)

charger les data brutes et renommage des colone selon la doc

In [6]:
_raw_train = pd.read_csv(train_path, sep=r'\s+', header=None)
_raw_test  = pd.read_csv(test_path,  sep=r'\s+', header=None)

n_cols = _raw_train.shape[1]
use_cols = min(n_cols, 31)            
n_sensors = use_cols - 5              

cols = (["unit","cycle"] +
        [f"setting_{i}" for i in range(1,4)] +
        [f"sensor_{i}" for i in range(1, n_sensors+1)])

train = _raw_train.iloc[:, :use_cols].copy()
test  = _raw_test.iloc[:,  :use_cols].copy()
train.columns = cols
test.columns  = cols

train.head(), test.head(), len(cols), n_sensors


(   unit  cycle  setting_1  setting_2  setting_3  sensor_1  sensor_2  sensor_3  \
 0     1      1    -0.0007    -0.0004      100.0    518.67    641.82   1589.70   
 1     1      2     0.0019    -0.0003      100.0    518.67    642.15   1591.82   
 2     1      3    -0.0043     0.0003      100.0    518.67    642.35   1587.99   
 3     1      4     0.0007     0.0000      100.0    518.67    642.35   1582.79   
 4     1      5    -0.0019    -0.0002      100.0    518.67    642.37   1582.85   
 
    sensor_4  sensor_5  ...  sensor_12  sensor_13  sensor_14  sensor_15  \
 0   1400.60     14.62  ...     521.66    2388.02    8138.62     8.4195   
 1   1403.14     14.62  ...     522.28    2388.07    8131.49     8.4318   
 2   1404.20     14.62  ...     522.42    2388.03    8133.23     8.4178   
 3   1401.87     14.62  ...     522.86    2388.08    8133.83     8.3682   
 4   1406.22     14.62  ...     522.19    2388.04    8133.80     8.4294   
 
    sensor_16  sensor_17  sensor_18  sensor_19  sensor

In [6]:
train.head()

,unit,cycle,setting_1,setting_2,setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [7]:
test.head()

,unit,cycle,setting_1,setting_2,setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


construction de la cible RUL sur le train. ligne pas ligne en applicant la formule RUL = max_cycle(unit) - cycle

In [7]:
max_cycle_by_unit = train.groupby("unit")["cycle"].max().rename("max_cycle")
train = train.merge(max_cycle_by_unit, left_on="unit", right_index=True)
train["RUL"] = train["max_cycle"] - train["cycle"]
train.drop(columns=["max_cycle"], inplace=True)

train[["unit","cycle","RUL"]].head(8)

,unit,cycle,RUL
0,1,1,191
1,1,2,190
2,1,3,189
3,1,4,188
4,1,5,187
5,1,6,186
6,1,7,185
7,1,8,184


suppresion des colonne constant (variane = 0 )

In [8]:
variances = train.var(numeric_only=True)
constant_cols = variances[(variances == 0) | (variances < 1e-12)].index.tolist()

PROTECT = {"unit","cycle","RUL"}
constant_cols = [c for c in constant_cols if c not in PROTECT]

constant_cols

['setting_3',
 'sensor_1',
 'sensor_5',
 'sensor_10',
 'sensor_16',
 'sensor_18',
 'sensor_19']

je vais maintenant supprimer et jounalier pour la tracabiliter

In [9]:
train = train.drop(columns=constant_cols)
test = test.drop(columns=[c for c in constant_cols if c in test.columns])

#journal de tracabiliter
(pd.Series(constant_cols)
   .to_frame("dropped_constant_columns")
   .to_csv(DOCS_DIR / "dropped_sensors_v1.csv", index=False))

len(constant_cols), "colonnes supprimées (constants)"

(7, 'colonnes supprimées (constants)')

In [11]:
train.head()

,unit,cycle,setting_1,setting_2,sensor_2,sensor_3,sensor_4,sensor_6,sensor_7,sensor_8,sensor_9,sensor_11,sensor_12,sensor_13,sensor_14,sensor_15,sensor_17,sensor_20,sensor_21,RUL
0,1,1,-0.0007,-0.0004,641.82,1589.70,1400.60,21.61,554.36,2388.06,9046.19,47.47,521.66,2388.02,8138.62,8.4195,392,39.06,23.4190,191
1,1,2,0.0019,-0.0003,642.15,1591.82,1403.14,21.61,553.75,2388.04,9044.07,47.49,522.28,2388.07,8131.49,8.4318,392,39.00,23.4236,190
2,1,3,-0.0043,0.0003,642.35,1587.99,1404.20,21.61,554.26,2388.08,9052.94,47.27,522.42,2388.03,8133.23,8.4178,390,38.95,23.3442,189
3,1,4,0.0007,0.0000,642.35,1582.79,1401.87,21.61,554.45,2388.11,9049.48,47.13,522.86,2388.08,8133.83,8.3682,392,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,642.37,1582.85,1406.22,21.61,554.00,2388.06,9055.15,47.28,522.19,2388.04,8133.80,8.4294,393,38.90,23.4044,187


normalisation pas uniter (Z-score):
chaque moteur a ces niveau normaux. on veux donc mettre toutes les colonnes sur la meme echelle a l'interieur de chque uniter.
pour chaque colonnes numerique x et pour chaque unit, on applique:
z = (x-mean(unit)/ std(unit)).

In [10]:
NUMERIC = [c for c in train.columns if c not in {"unit", "cycle","RUL"}]

def zscore_by_unit(df, numeric_cols):
    #on groupe pas uniter pour chaque colonnes
    out = df.copy()
    grp = out.groupby("unit")
    for c in numeric_cols:
        out[c] = grp[c].transform(lambda s: (s - s.mean()) / (s.std(ddof=0) if s.std(ddof=0) != 0 else 1.0))
    return out

train_norm = zscore_by_unit(train, NUMERIC)
test_norm = zscore_by_unit(test, [c for c in NUMERIC if c in test.columns])

train_norm.head(), test_norm.head()

(   unit  cycle  setting_1  setting_2  sensor_2  sensor_3  sensor_4  sensor_6  \
 0     1      1  -0.262241  -1.592766 -1.649843  0.037335 -0.779811      -1.0   
 1     1      2   1.072220  -1.237585 -0.970168  0.406367 -0.482501      -1.0   
 2     1      3  -2.109955   0.893503 -0.558244 -0.260328 -0.358426      -1.0   
 3     1      4   0.456315  -0.172041 -0.558244 -1.165501 -0.631156      -1.0   
 4     1      5  -0.878146  -0.882403 -0.517051 -1.155057 -0.121983      -1.0   
 
    sensor_7  sensor_8  sensor_9  sensor_11  sensor_12  sensor_13  sensor_14  \
 0  1.013802 -0.710609 -0.423722  -0.164630   0.268425  -1.183350   1.749346   
 1  0.342026 -0.993380 -0.856459  -0.089904   1.098164  -0.531965   0.464342   
 2  0.903675 -0.427838  0.954097  -0.911884   1.285524  -1.053073   0.777933   
 3  1.112917 -0.003682  0.247837  -1.434963   1.874371  -0.401688   0.886068   
 4  0.617344 -0.710609  1.405206  -0.874522   0.977718  -0.922796   0.880662   
 
    sensor_15  sensor_17  sens

fenetrage temporel --W = 30 cycle: Source du Code: Copilot IA

1- fonction d'extraction de features pas fenetre

In [24]:
from numpy.polynomial.polynomial import polyfit
WINDOW = 30

def window_feature(df, numeric_cols, window=30):
    ### pour chaque uniter, on vas calculer des features de fenetre glissante au cycl t:
    #-mean, std, min, max
    #-slope (pente) via une regresson lineaire simple (valeur ~ cycle)
    # on vas retouner un Dataframe aligner sur les ligne ou la fenetre est complete

    all_frames = []
    for u, g in df.sort_values(["unit", "cycle"]).groupby("unit"):
        g = g.reset_index(drop=True)
        feats = {}
        feats["unit"] =  g["unit"]
        feats["cycle"] = g["cycle"]
        #pour chaque capteur, on calcule les stats glissante
        for c in numeric_cols:
            s = g[c]
            # on calcule mean/std/min/max; on algne sur la fin de la fenetre
            feats[f"{c}_mean_W{window}"] = s.rolling(window, min_periods=window).mean()
            feats[f"{c}_std_W{window}"]  = s.rolling(window, min_periods=window).std(ddof=0)
            feats[f"{c}_min_W{window}"]  = s.rolling(window, min_periods=window).min()
            feats[f"{c}_max_W{window}"]  = s.rolling(window, min_periods=window).max()

            #pente : regression lineaire y ~ x sur la fenetre
            def slope_last_w(vals):
                if len(vals) < window:
                    return np.nan
                x = np.arange(window) # 0..W-1
                b0, b1 = polyfit(x, np.asarray(vals), 1) # b0 + b1*x
                return b1
            feats[f"{c}_slope_W{window}"] = s.rolling(window, min_periods=window).apply(lambda v: slope_last_w(v), raw=False)
        all_frames.append(pd.DataFrame(feats))
    out = pd.concat(all_frames, ignore_index=True)
    return out
                    
        

2- on applique le fenetrage au train et aligner au RUL

In [ ]:
# Colonnes numériques dispo côté train_norm (sans unit/cycle/RUL)
NUMERIC_TRAIN = [c for c in train_norm.columns if c not in {"unit","cycle","RUL"}]

train_feats = window_feature(train_norm, NUMERIC_TRAIN, window=WINDOW)

# On fusionne la cible RUL sur (unit,cycle)
train_w = train_feats.merge(train_norm[["unit","cycle","RUL"]], on=["unit","cycle"], how="left")

# Retirer les lignes sans fenêtre complète (NaN)
train_w = train_w.dropna().reset_index(drop=True)

train_w.shape


2- on fait la meme chose sur nos data de test

In [ ]:
NUMERIC_TEST = [c for c in test_norm.columns if c not in {"unit","cycle"}]
test_w = window_feature(test_norm, NUMERIC_TEST, window=WINDOW)
test_w = test_w.dropna().reset_index(drop=True)

test_w.shape


slip par uniter (train / validation) on evite les fuite d'info: ici on veux que la validation reflete la generalisation a de nouveau moteur, pas de nouvelles ligne d'un meme moteur

In [ ]:
rng = np.random.default_rng(42)
units = train_w["unit"].unique()
rng.shuffle(units)

n_train_units = int(0.8* len(units)) #80% d'uniter pour l'entrainement
train_units = set(units[:n_train_units])
val_units = set(units[n_train_units:])

Xy_train = train_w[train_w["unit"].isin(train_units)].reset_index(drop=True)
Xy_val   = train_w[train_w["unit"].isin(val_units)].reset_index(drop=True)

Xy_train.shape, Xy_val.shape, len(train_units), len(val_units)

Sauvegarde du dataset et metadata

In [31]:
# Enregistrment
Xy_train.to_csv(PROC_DIR / "train_ready.csv", index=False)
Xy_val.to_csv(PROC_DIR / "val_ready.csv", index=False)
test_w.to_csv(PROC_DIR / "test_features.csv", index=False)

# Sauvegarder la liste des features (utile pour le README / rapport)
feature_cols = [c for c in Xy_train.columns if c not in {"unit","cycle","RUL"}]
pd.Series(feature_cols, name="feature").to_csv(DOCS_DIR / "feature_list_v1.csv", index=False)

# Journaliser les choix du preprocessing
with open(DOCS_DIR / "preprocessing_config_v1.txt", "w", encoding="utf-8") as f:
    f.write(
        f"Normalization = Z-score per unit\n"
        f"Window size   = {WINDOW}\n"
        f"Dropped const = {constant_cols}\n"
        f"Split units   = 80/20 (seed=42)\n"
    )

"Files saved:", [
    str(PROC_DIR / "train_ready.csv"),
    str(PROC_DIR / "val_ready.csv"),
    str(PROC_DIR / "test_features.csv"),
    str(DOCS_DIR / "feature_list_v1.csv"),
    str(DOCS_DIR / "preprocessing_config_v1.txt"),
    str(DOCS_DIR / "dropped_sensors_v1.csv"),
]


('Files saved:',
 ['C:\\Users\\Brydel\\Desktop\\RUL_Predictor\\data\\processed\\train_ready.csv',
  'C:\\Users\\Brydel\\Desktop\\RUL_Predictor\\data\\processed\\val_ready.csv',
  'C:\\Users\\Brydel\\Desktop\\RUL_Predictor\\data\\processed\\test_features.csv',
  'C:\\Users\\Brydel\\Desktop\\RUL_Predictor\\docs\\feature_list_v1.csv',
  'C:\\Users\\Brydel\\Desktop\\RUL_Predictor\\docs\\preprocessing_config_v1.txt',
  'C:\\Users\\Brydel\\Desktop\\RUL_Predictor\\docs\\dropped_sensors_v1.csv'])

controle rapide pour la doc

In [32]:
def quick_report(df, name):
    n_units = df["unit"].nunique()
    n_rows  = len(df)
    n_feat  = len([c for c in df.columns if c not in {"unit","cycle","RUL"}])
    print(f"{name}: rows={n_rows:,} | units={n_units} | features={n_feat}")

quick_report(Xy_train, "train_ready")
quick_report(Xy_val,   "val_ready")
quick_report(test_w,   "test_features (no target)")


train_ready: rows=14,207 | units=80 | features=85
val_ready: rows=3,524 | units=20 | features=85
test_features (no target): rows=10,196 | units=100 | features=86
